# Train

In [ ]:
from mylib.data import my_dataloader, my_dataset, my_preprocess
from mylib.classification import classification
from mylib.classification.models import my_cnn, my_model_utils

In [ ]:
import sys
import pandas as pd
import torch

In [ ]:
from tqdm import tqdm
from functools import partialmethod

tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [ ]:
print(f"Python version: {sys.version}")
print(f"Torch version:  {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"device name:    {torch.cuda.get_device_name()}")

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## I/O

In [ ]:
image_path  = "data/images"
label_path  = "data/labels/label_v2.xlsx"
output_path = "outputs/02"

fname_key = "fname"
label_key = "pap2_groundtruth"
fold_key  = "slide"

name_classes = ['positive','negative']
num_classes  = len(name_classes)

## Parameters

In [ ]:
# 実験のパラメータ
num_epochs = 100
image_size = 224
batch_size = 16
val_ratio  = 0.1
lr = 1e-4  # 学習率

# データ拡張
# Normalize -> RandomFlip -> RandomAffine
transforms = my_preprocess.getStandardTransforms()


# 再学習レイヤー
# レイヤーには名前がついているので，再学習させるレイヤーの名前をリストに追加する
training_layers = ['fc']  # 全結合層のみ学習させる

## Data

In [ ]:
# データのロード
_images, _labels, _fnames, _folds = my_dataloader.loadData4(
    image_path,
    label_path,
    fname_key = fname_key,
    label_key = label_key,
    fold_key  = fold_key,
    resize    = image_size,
    to_tensor = True
)
print(len(_fnames))


# 不適切なデータを除去
# 不適切なデータ：label=-1
images, labels, fnames, folds = [], [], [], []
for i in range(len(_images)):
    if _labels[i] != -1:
        images.append(_images[i])
        labels.append(_labels[i])
        fnames.append(_fnames[i])
        folds.append(_folds[i])

In [ ]:
# データセットの作成
dataset = my_dataset.MyDataset(images, labels, fnames, name_classes)

# Leave-one-case-outマネージャーを作成
managers = my_dataset.getLocoManager(folds)

## Train

In [ ]:
for manager in managers:  # このループが 1fold
    fold_id, fold_name = manager['fold_id'], manager['fold_name']
    print(f"======== fold-id: {fold_id} / fold-name: {fold_name} ========")
    
    # モデルを作成
    model, params = my_cnn.build_ResNet50(num_classes=num_classes, training_layers=training_layers)
    model.to(device)
    model.apply(my_model_utils.resetWeights)  # 重みをリセット
    
    # データローダーを作成
    loader = my_dataset.getLoader(dataset, manager, batch_size, val_ratio=val_ratio)
    
    # 損失関数
    criterion = torch.nn.CrossEntropyLoss()
    
    # オプティマイザ
    optimizer = torch.optim.SGD(params, lr=lr, momentum=0.9)
    
    # スケジューラ
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs)
    
    # 学習履歴保存用
    history = dict()
    history['train_loss'], history['valid_loss'], history['train_acc'], history['valid_acc'] = [], [], [], []
    
    # 学習
    best_loss = 1000.0  # 最良のlossを保存
    for epoch in range(1, num_epochs+1):  # このループが 1epoch
        ep_loss, ep_acc = classification.train(
            device = device,
            model  = model,
            train_loader = loader['train'],
            valid_loader = loader['valid'],
            num_classes  = num_classes,
            criterion    = criterion,
            optimizer    = optimizer,
            scheduler    = scheduler,
            transforms   = transforms,
        )
        
        # 学習履歴の保存
        history['train_loss'].append(ep_loss['train'])
        history['valid_loss'].append(ep_loss['valid'])
        history['train_acc'].append(ep_acc['train'])
        history['valid_acc'].append(ep_acc['valid'])
        
        # 最良のlossの場合モデルを保存
        if ep_loss['valid'] < best_loss:
            torch.save(model.state_dict(), f"{output_path}/tmp.pth")
            best_loss = ep_loss['valid']
            
        # ログ出力
        current_lr = optimizer.param_groups[0]['lr']
        log = f"\repoch {epoch}/{num_epochs} (lr={current_lr:.6f}): <train> loss={ep_loss['train']:.4f} acc={ep_acc['train']:.4f} <valid> loss={ep_loss['valid']:.4f} acc={ep_acc['valid']:.4f}"
        print(log, end="")
        
    # チェックポイントを生成
    model.load_state_dict(torch.load(f"{output_path}/tmp.pth"))  # 最良モデルをロード
    torch.save(model.state_dict(), f"{output_path}/ckpt/{fold_name}_model.pth")
    
    # 学習履歴ファイルを生成
    df_history = pd.DataFrame.from_dict(history)
    df_history.to_excel(f"{output_path}/history/{fold_name}_history.xlsx", index=False)